In [2]:
import os
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
from dotenv import load_dotenv

def init_spotify_client(client_id: str, client_secret: str) -> spotipy.Spotify:
    auth_manager = SpotifyClientCredentials(client_id=client_id, client_secret=client_secret)
    return spotipy.Spotify(auth_manager=auth_manager)

def extract_artists(sp: spotipy.Spotify, artists: list) -> list[dict]:
    artist_data = []
    for artist_name in artists:
        results = sp.search(q=f'artist:{artist_name}', type='artist', limit=1)
        artist_items = results['artists']['items']
        if artist_items:
            artist_info = artist_items[0]
            artist_data.append({
                'name': artist_info['name'],
                'id': artist_info['id']
            })
    return artist_data


def get_artist_followers(sp: spotipy.Spotify, artist_id: str) -> int:
    """
    Gets the number of followers of an artist.

    :param sp: Spotify client object
    :param artist_id: The Spotify ID of the artist
    :return: Number of followers
    """
    artist_info = sp.artist(artist_id)
    return artist_info['followers']['total']

# Load environment variables
load_dotenv()
CLIENT_ID = os.environ.get("SPOTIFY_CLIENT_ID")
CLIENT_SECRET = os.environ.get("SPOTIFY_CLIENT_SECRET")

# Initialize Spotify client
sp = init_spotify_client(CLIENT_ID, CLIENT_SECRET)

# Example usage
artists = ["Radiohead", "Coldplay"]  # Replace with your list of artists
artist_info = extract_artists(sp, artists)

# Get followers for each artist
for artist in artist_info:
    followers = get_artist_followers(sp, artist['id'])
    print(f"Artist: {artist['name']}, Followers: {followers}")


Artist: Radiohead, Followers: 9236966
Artist: Coldplay, Followers: 48825652


In [8]:
import os
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
from dotenv import load_dotenv

def init_spotify_client(client_id: str, client_secret: str) -> spotipy.Spotify:
    auth_manager = SpotifyClientCredentials(client_id=client_id, client_secret=client_secret)
    return spotipy.Spotify(auth_manager=auth_manager)
"""
def extract_artists(sp:  spotipy.Spotify, artists: list) -> list[dict]:
    artist_data = []
    for artist_name in artists:
        results = sp.search(q=f'artist:{artist_name}', type='artist', limit=1)
        artist_items = results['artists']['items']
        if artist_items:
            artist_data.append(artist_items[0])
    return artist_data

"""
def extract_artists(sp: spotipy.Spotify, artists: list, date: str) -> list[dict]:
    artist_data = []
    for artist_name in artists:
        results = sp.search(q=f'artist:{artist_name} year:{date}', type='artist', limit=1)
        artist_items = results['artists']['items']
        if artist_items:
            artist_data.append(artist_items[0])
    return artist_data

# Load environment variables
load_dotenv()
CLIENT_ID = os.environ.get("SPOTIFY_CLIENT_ID")
CLIENT_SECRET = os.environ.get("SPOTIFY_CLIENT_SECRET")

# Initialize Spotify client
sp = init_spotify_client(CLIENT_ID, CLIENT_SECRET)


date = "2023"  # Replace with your desired date
# Example usage
artists = ["Radiohead", "Coldplay"]  # Replace with your list of artists
artist_info = extract_artists(sp,date, artists)



In [13]:
import os
import pandas as pd
import psycopg2
import json
import datetime
from spotipy import Spotify
from spotipy.oauth2 import SpotifyClientCredentials
from sqlalchemy import create_engine, Table, Column, String, Integer, MetaData, ForeignKey, Boolean
from dotenv import load_dotenv
from spotify_helpers import get_global_top_40_artists
from sqlalchemy import DateTime

def init_spotify_client(client_id: str, client_secret: str) -> Spotify:
    auth_manager = SpotifyClientCredentials(client_id=client_id, client_secret=client_secret)
    return Spotify(auth_manager=auth_manager)

def extract_global_playlist_countries(sp, markets):
    global_playlist = []
    for market in markets:
        # Fetch top tracks for each market
        top_tracks = sp.playlist_tracks('37i9dQZEVXbMDoHDwVN2tF', market=market)

        # Extracting required info from each track
        for track in top_tracks['items']:
            # Assuming track data structure contains 'track' which has 'artists' and 'id'
            artist_id = track['track']['artists'][0]['id']  # ID of the first artist
            playlist_country = market

            # Append the information as a list to the global_playlist
            global_playlist.append([artist_id, playlist_country])

    return global_playlist



def nomarket(sp):
    global_playlist=[]
    # Fetch top tracks for each market
    top_tracks = sp.playlist_tracks('37i9dQZEVXbMDoHDwVN2tF')
    # Extracting required info from each track
    for track in top_tracks['items']:
        # Assuming track data structure contains 'track' which has 'artists' and 'id'
        artist_id = track['track']['artists'][0]['id']  # ID of the first artist

        # Append the information as a list to the global_playlist
        global_playlist.append([artist_id])

    return global_playlist

markets=["US","ES","PT","UK","BE"]

In [14]:
load_dotenv()
CLIENT_ID = os.environ.get("SPOTIFY_CLIENT_ID")
CLIENT_SECRET = os.environ.get("SPOTIFY_CLIENT_SECRET")
DB_USERNAME = os.environ.get("DB_USERNAME")
DB_PASSWORD = os.environ.get("DB_PASSWORD")
SERVER_NAME = os.environ.get("SERVER_NAME")
DATABASE_NAME = os.environ.get("DATABASE_NAME")

sp = init_spotify_client(CLIENT_ID, CLIENT_SECRET)

In [21]:
markets=["US","ES","PT","UK","BE"]

list_us=extract_global_playlist_countries(sp,markets)

In [18]:
list_us_nomarket=nomarket(sp)

In [33]:
import pandas as pd

# Assuming list_us and list_us_nomarket are defined
# Example: list_us = [[1, 'US'], [2, 'UK'], [3, 'US']]
# Example: list_us_nomarket = [[1], [3]]

# Create DataFrames from the lists
df_us = pd.DataFrame(list_us)
df_us_nomarket = pd.DataFrame(list_us_nomarket)

# Filter rows in df_us where the second column equals 'US'
filtered_df_us = df_us[df_us[1] == 'US']

# Compare the values in the first column of filtered_df_us with df_us_nomarket
result = filtered_df_us[0].values == df_us_nomarket.values.flatten()

print(result)


[ True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True]


In [35]:
def extract_global_playlist_countries(sp, markets):
    global_playlist = []
    for market in markets:
        top_tracks = sp.playlist_tracks('37i9dQZEVXbMDoHDwVN2tF', market=market)
        for track_item in top_tracks['items']:
            track = track_item['track']
            global_playlist.append((track, market))  # Appending as a tuple
    return global_playlist



def transform_playlist(playlist_data):
    # Normalize and flatten the data
    tracks = [item[0] for item in playlist_data]  # Extracting track details
    countries = [item[1] for item in playlist_data]  # Extracting country details
    df_tracks = pd.json_normalize(tracks)
    
    # Add the country data
    df_tracks['playlist_country'] = countries
    
    # Select and rename columns
    df_selected = df_tracks[['id', 'playlist_country']]
    df_selected = df_selected.rename(columns={"id": "artist_id"})
    return df_selected

In [39]:
markets=["US","ES","PT","BE","GB"]



playlist_data = extract_global_playlist_countries(sp, markets)
df_playlist = transform_playlist(playlist_data)

In [25]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
from collections import Counter
import csv

def get_global_top_40_artists(client_id, client_secret, csv_filename,market="US"):
    # Authentication
    client_credentials_manager = SpotifyClientCredentials(client_id=client_id, client_secret=client_secret)
    sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)



    


    # Fetch top tracks globally (you can adjust the market if needed)
    top_tracks = sp.playlist_tracks('37i9dQZEVXbMDoHDwVN2tF',market=market)  # This is the Global Top 50 playlist ID

    

    """
    print(top_tracks)

    # Extract artist IDs from the tracks
    artist_ids = [track['track']['artists'][0]['id'] for track in top_tracks['items']]

    # Count the frequency of each artist ID
    artist_counts = Counter(artist_ids)
    """
    return top_tracks



import requests
from collections import Counter

def get_global_top_40_artists(client_id, client_secret, csv_filename,playlist_id):
    # Spotify API URL
    api_url = 'https://api.spotify.com/v1'
    
    # Get access token
    auth_response = requests.post(
        'https://accounts.spotify.com/api/token',
        data={
            'grant_type': 'client_credentials',
            'client_id': client_id,
            'client_secret': client_secret,
        }
    )

    # Extract the access token from the response
    access_token = auth_response.json()['access_token']

    # Set up the header for subsequent API calls
    headers = {
        'Authorization': f'Bearer {access_token}',
    }

    # Fetch top tracks globally (you can adjust the market if needed)
    # This is the Global Top 50 playlist ID
    playlist_id = playlist_id
    response = requests.get(f'{api_url}/playlists/{playlist_id}/tracks', headers=headers)

    # Parse the JSON response
    top_tracks = response.json()

    # [Your existing code to process top_tracks goes here]

    return top_tracks







response=get_global_top_40_artists("81cfd4bb5c0244d9b27bdfec12fb8bda", "b9a6cc52074740b7b9ea6260a81e25c1", 'top_40_artists.csv',playlist_id="37i9dQZEVXbMDoHDwVN2tF")


# Function to extract artist names and IDs
def get_artist_names_and_ids(response):
    artist_info = []
    for item in response.get('items', []):
        track = item.get('track', {})
        artists = track.get('artists', [])
        for artist in artists:
            name = artist.get('name')
            artist_id = artist.get('id')
            if name and artist_id:
                artist_info.append(artist_id)
    return artist_info

# Extracting artist names and IDs
artist_info = get_artist_names_and_ids(response)
print("Artist Names and IDs:", artist_info)


Artist Names and IDs: ['66CXWjxzNUsdJxJ2JdwvnR', '3Me35AWHCGqW4sZ7bWWJt1', '45dkTj5sMRSjrmBSBeiHym', '2LIk90788K0zvyj2JJVwkJ', '2uYWxilOVlUdk4oV9DvwqK', '1Xyo4u8uXC1ZmMpatF05PJ', '250b0Wlc5Vk0CoUsaCY84M', '1pBLC0qVRTB5zVMuteQ9jJ', '06HL4z0CvFAxyc27GXpf02', '1URnnhqYAYcrqrcwql10ft', '2RQXRUsr4IW1f3mKyKsy4B', '3Me35AWHCGqW4sZ7bWWJt1', '2LRoIwlKmHjgvigdNGBHNo', '7jAs1uPnpfNmT6e5qtEPxq', '1U1el3k54VvEUzo3ybLPlM', '12GqGscKJx3aE4t07u7eVZ', '716NhGYqD1jl2wI1Qkgq36', '3qsKSpcV3ncke3hw52JSMB', '1Xyo4u8uXC1ZmMpatF05PJ', '699OTQXzgjhIYAHMy9RyPD', '6tbjWDEIzxoDsBA1FuhfPW', '1URnnhqYAYcrqrcwql10ft', '0Y5tJX1MQlPlqiwlOH1tJY', '0iEtIxbK0KxaSlF7G42ZOp', '7Ln80lUS6He07XvHI8qqHH', '12GqGscKJx3aE4t07u7eVZ', '7FNnA9vBm6EKceENgCGRMb', '6HaGTQPmzraVmaVxvz6EUc', '3MdXrJWsbVzdn6fe5JYkSQ', '2cBh5lVMg222FFuRU7EfDE', '5cj0lLjcoR7YOSnhnX0Po5', '4q3ewBCX7sLwd24euuV69X', '2LRoIwlKmHjgvigdNGBHNo', '7uMDnSZyUYNBPLhPMNuaM2', '1McMsnEElThX1knmY4oliG', '790FomKkXshlbRYZFtlgla', '12GqGscKJx3aE4t07u7eVZ', '6HaGTQPmzraVma

In [ ]:

{"global":"37i9dQZEVXbMDoHDwVN2tF","SW":"7jmQBEvJyGHPqKEl5UcEe9","PT":"7i9dQZEVXbKyJS56d1pgi","UK":"37i9dQZEVXbLnolsZ8PSNw","USA":"7i9dQZEVXbLRQDuF5jeBp","BE":"37i9dQZEVXbJNSeeHswcKB"}

def extract_global_playlist_countries(sp, playlist_id):
    global_playlist = []
    for market in markets:
        top_tracks = sp.playlist_tracks(playlist_id)
        for track_item in top_tracks['items']:
            track = track_item['track']
            global_playlist.append((track,country))  # Appending as a tuple
    return global_playlist

def transform_playlist(playlist_data):
    # Normalize and flatten the data
    tracks = [item[0] for item in playlist_data]  # Extracting track details
    countries = [item[1] for item in playlist_data]  # Extracting country details
    df_tracks = pd.json_normalize(tracks)
    
    # Add the country data
    df_tracks['playlist_country'] = countries
    
    # Select and rename columns
    df_selected = df_tracks[['id', 'playlist_country']]
    df_selected = df_selected.rename(columns={"id": "artist_id"})
    return df_selected

In [62]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
from collections import Counter
import csv
import os
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
from dotenv import load_dotenv
import pandas as pd
# Load environment variables
def init_spotify_client(client_id: str, client_secret: str) -> spotipy.Spotify:
    auth_manager = SpotifyClientCredentials(client_id=client_id, client_secret=client_secret)
    return spotipy.Spotify(auth_manager=auth_manager)

load_dotenv()
CLIENT_ID = os.environ.get("SPOTIFY_CLIENT_ID")
CLIENT_SECRET = os.environ.get("SPOTIFY_CLIENT_SECRET")

# Initialize Spotify client
sp = init_spotify_client(CLIENT_ID, CLIENT_SECRET)


def extract_global_playlist_countries(sp, playlist_dict):
    global_playlist = []
    for country, playlist_id in playlist_dict.items():
        top_tracks = sp.playlist_tracks(playlist_id)
        for track_item in top_tracks['items']:
            track = track_item['track']
            global_playlist.append((track, country))  # Appending as a tuple
    return global_playlist

# Dictionary of playlist IDs
playlist_dict = {
    "global": "37i9dQZEVXbMDoHDwVN2tF",
    "SW": "7jmQBEvJyGHPqKEl5UcEe9",
    "UK": "37i9dQZEVXbLnolsZ8PSNw",
    "BE": "37i9dQZEVXbJNSeeHswcKB",
    "USA":"37i9dQZEVXbLRQDuF5jeBp",
    "PT":"37i9dQZEVXbKyJS56d1pgi"
}

playlist_data = extract_global_playlist_countries(sp, playlist_dict)


def transform_playlist(playlist_data):
    # Initialize an empty list to hold artist IDs and countries
    artist_data = []

    # Loop through each track in the playlist data
    for track, country in playlist_data:
        # Each track can have multiple artists
        for artist in track['artists']:
            # Extract the artist ID and append it along with the country to the list
            artist_data.append((artist['id'], country))

    # Convert the list of tuples into a DataFrame
    df_artists = pd.DataFrame(artist_data, columns=['artist_id', 'playlist_country'])

    return df_artists

# Now call the function with your playlist data
transformed_df = transform_playlist(playlist_data)



In [71]:
transformed_df

,artist_id,playlist_country
0,66CXWjxzNUsdJxJ2JdwvnR,global
1,3Me35AWHCGqW4sZ7bWWJt1,global
2,45dkTj5sMRSjrmBSBeiHym,global
3,2LIk90788K0zvyj2JJVwkJ,global
4,2uYWxilOVlUdk4oV9DvwqK,global
...,...,...
492,1pPbDeOdNUcLq32HTTLbZm,PT
493,3qvcCP2J0fWi0m0uQDUf6r,PT
494,2CKOmarVWvWqkNWUatHCex,PT
495,7me0S5Z40qVWj3gzyK8aC3,PT


In [69]:
transformed_df[transformed_df["playlist_country"]=="global"]["artist_id"].to_csv("top_artist_from_global50")

0     66CXWjxzNUsdJxJ2JdwvnR
1     3Me35AWHCGqW4sZ7bWWJt1
2     45dkTj5sMRSjrmBSBeiHym
3     2LIk90788K0zvyj2JJVwkJ
4     2uYWxilOVlUdk4oV9DvwqK
               ...          
71    66ihevNkSYNzRAl44dx6jJ
72    1Xyo4u8uXC1ZmMpatF05PJ
73    6M2wZ9GZgrQXHCFfjv46we
74    5YGY8feqx7naU7z4HrwZM6
75    1oSPZhvZMIrWW5I41kPkkY
Name: artist_id, Length: 76, dtype: object

In [70]:

# Extracting unique artist IDs where playlist_country is "global"
unique_artists_global = transformed_df[transformed_df["playlist_country"] == "global"]["artist_id"].unique()

# Converting to DataFrame to use to_csv
unique_artists_df = pd.DataFrame(unique_artists_global, columns=["artist_id"])
unique_artists_df.to_csv("top_artist_from_global50.csv", index=False)

unique_artists_df.head()

,artist_id
0,66CXWjxzNUsdJxJ2JdwvnR
1,3Me35AWHCGqW4sZ7bWWJt1
2,45dkTj5sMRSjrmBSBeiHym
3,2LIk90788K0zvyj2JJVwkJ
4,2uYWxilOVlUdk4oV9DvwqK


In [60]:
transformed_df[transformed_df["artist_id"]=="66CXWjxzNUsdJxJ2JdwvnR"]

,artist_id,playlist_country
0,66CXWjxzNUsdJxJ2JdwvnR,global
154,66CXWjxzNUsdJxJ2JdwvnR,UK
237,66CXWjxzNUsdJxJ2JdwvnR,BE
311,66CXWjxzNUsdJxJ2JdwvnR,USA
390,66CXWjxzNUsdJxJ2JdwvnR,PT


In [61]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials

def get_artist_details(sp,artist_id):
    # Set up your Spotify API credentials

    # Fetch artist details
    artist = sp.artist(artist_id)
    return artist

# Example usage
artist_id = '3Me35AWHCGqW4sZ7bWWJt1'
artist_details = get_artist_details(sp,artist_id)
print(artist_details)


{'external_urls': {'spotify': 'https://open.spotify.com/artist/3Me35AWHCGqW4sZ7bWWJt1'}, 'followers': {'href': None, 'total': 1366379}, 'genres': ['sad sierreno'], 'href': 'https://api.spotify.com/v1/artists/3Me35AWHCGqW4sZ7bWWJt1', 'id': '3Me35AWHCGqW4sZ7bWWJt1', 'images': [{'height': 640, 'url': 'https://i.scdn.co/image/ab6761610000e5ebf25379218c13e6aef1f1793d', 'width': 640}, {'height': 320, 'url': 'https://i.scdn.co/image/ab67616100005174f25379218c13e6aef1f1793d', 'width': 320}, {'height': 160, 'url': 'https://i.scdn.co/image/ab6761610000f178f25379218c13e6aef1f1793d', 'width': 160}], 'name': 'Xavi', 'popularity': 84, 'type': 'artist', 'uri': 'spotify:artist:3Me35AWHCGqW4sZ7bWWJt1'}
